In [43]:
import cobra.test
import pandas
import os
from os.path import join
from cobra import Model, Reaction, Metabolite
from cobra.io import save_json_model
from cobra.io import write_sbml_model
from cobra.io import read_sbml_model

data_dir = 'C:\\Users\\alexa\\Documents\\Research\\Metabolic models\\COBRA\\COBRApy'

model = read_sbml_model("../Data/Models/Av_annotate.xml")


## Rnf reversibility and directionality

Rnf has been characterized in aceotgens but not in any aerobic nitrogen fixer. We have made an argument for RNF stoichiometry in the reversible direction (pmf consuming) to be different and require at least 5 protons to reduce ferredoxin in the conditions present in *A. vinelandii*. Please refer to the markdown document Rnf_stoich.md in the notebook folder.

In [44]:
#chanage the direction of Rnf to reduced Fd and oxidize NADH
RNF = model.reactions.get_by_id("RNF")

RNF.subtract_metabolites({model.metabolites.get_by_id("h_e"): 2})
RNF.add_metabolites({model.metabolites.get_by_id("h_p"): -5})
RNF.add_metabolites({model.metabolites.get_by_id("h_c"): 8})
RNF.add_metabolites({model.metabolites.get_by_id("fdxo_42_c"): -1})
RNF.add_metabolites({model.metabolites.get_by_id("fdxr_42_c"): 1})
RNF.add_metabolites({model.metabolites.get_by_id("nadh_c"): -1})
RNF.add_metabolites({model.metabolites.get_by_id("nad_c"): 1})

#change subsystem to nitrogen fixation

RNF.subsystem = "Nitrogen Fixation"

#clean up gene reaction rule

RNF.gene_reaction_rule = '( Avin_19210 or Avin_50920 ) and ( Avin_19220 or Avin_50930 ) and ( Avin_19230 or Avin_50940 ) and ( Avin_19240 or Avin_50950 ) and ( Avin_19250 or Avin_50960 ) and ( Avin_19260 or Avin_50970 ) and ( Avin_19270  or Avin_50980)'


In [45]:
#Set lower bound to 0 to make irreversable (can change to see effects later)
model.reactions.get_by_id("RNF").lower_bound = 0

In [46]:
RNF.gene_reaction_rule

'( Avin_19210 or Avin_50920 ) and ( Avin_19220 or Avin_50930 ) and ( Avin_19230 or Avin_50940 ) and ( Avin_19240 or Avin_50950 ) and ( Avin_19250 or Avin_50960 ) and ( Avin_19260 or Avin_50970 ) and ( Avin_19270  or Avin_50980)'

## Add Fix to the model

In [47]:
FIX = Reaction('FIX')
FIX.name = 'Fix NADH Quinone ferrdoxin oxioreductase'
FIX.lower_bound = 0.  # This is the default
FIX.upper_bound = 1000.  # This is the default

fdxo_42_c = model.metabolites.get_by_id("fdxo_42_c")
nadh_c = model.metabolites.get_by_id("nadh_c")
q8_c = model.metabolites.get_by_id("q8_c")
fdxr_42_c = model.metabolites.get_by_id("fdxr_42_c")
nad_c = model.metabolites.get_by_id("nad_c")
q8h2_c = model.metabolites.get_by_id("q8h2_c")

FIX.add_metabolites({
    fdxo_42_c: -1.0,
    nadh_c: -2.0,
    q8_c: -1.0,
    fdxr_42_c: 1.0,
    nad_c: 2.0,
    q8h2_c: 1.0
})

FIX.gene_reaction_rule = '(Avin_10520 and Avin_10530 and Avin_10540 and Avin_10550)'

#No annotation for bifurcating fix so this is annotation for ETF its homolog check ledbetter et al for biochemical details 10.1021/acs.biochem.7b00389
FIX.annotation.update({'ec-code': ['1.5.5.1'],
                     'kegg.reaction': 'R04433',
                      'seed.reaction': 'rxn17250'
                      })

FIX.subsystem = 'Nitrogen fixation'

model.add_reactions([FIX])

## Add NADH Dehydrogeanse I (coupled)

In [48]:
NDHI = Reaction('NADH6')
NDHI.name = 'NADH: quinone oxidoreductase'
NDHI.lower_bound = -1000.  # This is the default
NDHI.upper_bound = 1000.  # This is the default

#Just add metabolites not added before
h_c = model.metabolites.get_by_id("h_c")
h_p = model.metabolites.get_by_id("h_p")

#stochiometry of 4 protons translocated with extra proton for quniol production is from Berstova et al (Biochimica et Biophysica Acta 1363, 1998. 125–133)
NDHI.add_metabolites({
    h_c : -5.0,
    nadh_c: -1.0,
    q8_c: -1.0,
    h_p : 4.0,
    nad_c: 1.0,
    q8h2_c: 1.0
})


NDHI.gene_reaction_rule = '((Avin_28540) and (Avin_28440) and (Avin_28560) and (Avin_28450) and (Avin_28490) and (Avin_28510) and (Avin_28460) and (Avin_28520) and (Avin_28470))'

NDHI.annotation.update({'ec-code': ['7.1.1.2'],
                        'bigg.reaction': 'NADH6',
                        'kegg.reaction': 'R11945',
                        'metanetx.reaction': 'MNXR101873',
                        'seed.reaction': 'rxn10122'                        
                      })



model.add_reactions([NDHI])


## Remove nuo subunit genes from NADH II GRP rules 

In [49]:
NADH5 = model.reactions.get_by_id('NADH5')
NADH5.gene_reaction_rule = '(Avin_12000)'

## Add Cytochrome c oxioreductase (quinone)

In [50]:
QCCOR = Reaction('QCCOR')
QCCOR.name = 'quinone: cytochrome c oxidoreductase'
QCCOR.lower_bound = -1000.  # This is the default
QCCOR.upper_bound = 1000.  # This is the default

#Just add metabolites not added before
ficytC_c = model.metabolites.get_by_id("ficytC_c")
focytC_c = model.metabolites.get_by_id("focytC_c")

QCCOR.add_metabolites({
    
    ficytC_c: -2.0,
    q8h2_c: -1.0,
    h_p : 4.0,
    h_c : -2.0,
    focytC_c: 2.0,
    q8_c: 1.0
})

QCCOR.gene_reaction_rule = '(Avin_13060 and Avin_13070 and Avin_13080)'

QCCOR.annotation.update({
                        'metanetx.reaction': 'MNXR96964'                 
                      })

model.add_reactions([QCCOR])

## Add Alternative Nitorgnease (V and Fe)

In [51]:
VNIT = Reaction('VNIT')
VNIT.name = 'Vanadium Nitrogenase'
VNIT.lower_bound = 0.  # This is the default
VNIT.upper_bound = 1000.  # This is the default

fdxo_42_c = model.metabolites.get_by_id("fdxo_42_c")
n2_c = model.metabolites.get_by_id("n2_c")
atp_c = model.metabolites.get_by_id("atp_c")
fdxr_42_c = model.metabolites.get_by_id("fdxr_42_c")
adp_c = model.metabolites.get_by_id("adp_c")
h2o_c = model.metabolites.get_by_id("h2o_c")
h_c = model.metabolites.get_by_id("h_c")
pi_c = model.metabolites.get_by_id("pi_c")
nh4_c = model.metabolites.get_by_id("nh4_c")
h2_c = model.metabolites.get_by_id("h2_c")

VNIT.add_metabolites({
    fdxr_42_c: -6.0,
    n2_c: -1.0,
    atp_c: -24.0,
    h2o_c: -24.0,
    fdxo_42_c: 6.0,
    nh4_c: 2.0,
    h2_c: 3.0,
    adp_c: 24.0,
    pi_c: 24.0,
    h_c: 10
})

VNIT.gene_reaction_rule = '(Avin_02590 and Avin_02600 and Avin_02610 and Avin_02660)'

#the alternative nitrogenases are not accuratley annotated anywhere! so this is the best we can do Kegg has vandium but the stoichometery is wrong
VNIT.annotation.update({'ec-code': ['1.18.6.2'],
                        'kegg.reaction': 'R12084'                  
                      })

VNIT.subsystem  = 'Nitrogen fixation'
model.add_reactions([VNIT])

In [52]:
FENIT = Reaction('FENIT')
FENIT.name = 'Fe-only Nitrogenase'
FENIT.lower_bound = 0.  # This is the default
FENIT.upper_bound = 1000.  # This is the default


FENIT.add_metabolites({
    fdxr_42_c: -10.0,
    n2_c: -1.0,
    atp_c: -40.0,
    h2o_c: -40.0,
    fdxo_42_c: 10.0,
    nh4_c: 2.0,
    h2_c: 7.0,
    adp_c: 40.0,
    pi_c: 40.0,
    h_c: 18
})


FENIT.gene_reaction_rule = '(Avin_48970 and Avin_48980 and Avin_48990 and Avin_49000)'

#no Fe-only nitrogenase annotated in any database 

FENIT.subsystem = 'Nitrogen fixation'

model.add_reactions([FENIT])

## Fix nif GRP to only include nifHDK subunits 
Can adjust to all biosyntheis genes later

In [53]:
NIT = model.reactions.get_by_id('NIT1b')
NIT.gene_reaction_rule = '(Avin_01380 and Avin_01390 and Avin_01400)'

NIT.subsystem = 'Nitrogen fixation'

## Make soluble hydrogenase *hoxHY* consume H2 only

In [54]:
NAD_H2 = model.reactions.get_by_id("NAD_H2")
NAD_H2.lower_bound = -1000
NAD_H2.upper_bound = 0

NAD_H2.gene_reaction_rule = '(Avin_04380 and Avin_04390 and Avin_04400 and Avin_04410)'

NAD_H2.annotation.update({'ec-code': ['1.12.1.2', '1.12.1.5'],
                        'bigg.reaction': 'NAD_H2',
                        'kegg.reaction': 'R00700',
                        'metanetx.reaction': 'MNXR101899',
                        'seed.reaction': 'rxn05887'                        
                      })



## Fix gene reaction rules for the Cytchrome C oxidase (*cta* and *coxAB*) and add cytochrome cbb3 (*ccoNOQP*)

In [55]:
# cytochrome cbb3 (ccoNOQP) little is known about this and if cytochrome c is donor

CYTCBB3pp = Reaction('CYTCBB3pp')
CYTCBB3pp.name = 'Cytochrome c oxidase bb3 05 protons1 electron periplasm'
CYTCBB3pp.lower_bound = 0.  # This is the default
CYTCBB3pp.upper_bound = 0.  # This is set so we can compare bd/c as cbb3 is redudant

o2_c = model.metabolites.get_by_id("o2_c")

CYTCBB3pp.add_metabolites({
    h_c: -1.0,
    o2_c: -0.5,
    focytC_c: -2.0,
    ficytC_c: 2.0,
    h2o_c: 1.0,
    h_p: 1.0
})

CYTCBB3pp.gene_reaction_rule = '(Avin_19980 and Avin_19990 and Avin_20000 and Avin_20010)'

CYTCBB3pp.annotation.update({'ec-code': ['7.1.1.9'],
                     'kegg.reaction': 'R000081',
                      'seed.reaction': 'rxn14425'
                      })

CYTCBB3pp.subsystem = 'Oxidative Phosphorylation'

model.add_reactions([CYTCBB3pp])

In [56]:
#cytochrome o oxidase (coxAB and cta genes)

CYOO2pp = model.reactions.get_by_id("CYOO2pp")

CYOO2pp.gene_reaction_rule = '(Avin_11180 and Avin_11170) or (Avin_00990 and Avin_01010 and Avin_01020)'

CYOO2pp.annotation.update({'ec-code': ['7.1.1.9'],
                        'bigg.reaction': 'CYOO2pp',
                        'kegg.reaction': 'R00016',
                        'metanetx.reaction': 'MNXR138189',
                        'seed.reaction': 'rxn13688'                        
                      })

## Fix Cytochrome bd gpr
There are two cytbd the first (cydAB I, Avin_11170-80) has been heavily invesitagted (see ref in paper). The second Avin_11040-50 is not expressed but very close in the genome.

In [57]:
CYTBDpp = model.reactions.get_by_id("CYTBDpp")

CYTBDpp.gene_reaction_rule = '(Avin_11170 and Avin_11180) or (Avin_11040 and 11050)'

In [58]:
model.reactions.CYTBDpp.annotation

{'sbo': 'SBO:0000185',
 'bigg.reaction': 'CYTBDpp',
 'metanetx.reaction': 'MNXR97031',
 'seed.reaction': ['rxn12494', 'rxn10112', 'rxn08288']}

## Add a transhydrogenase for NADPH to NADH electron transfer

In [59]:
NADTRHD = Reaction('NADTRHD')
NADTRHD.name = 'NAD transhydrogenase'
NADTRHD.lower_bound = 0.  # This is the default
NADTRHD.upper_bound = 1000.  # This is the default

#Just add metabolites not added before
nadph_c = model.metabolites.get_by_id("nadph_c")
nadp_c = model.metabolites.get_by_id("nadp_c")


NADTRHD.add_metabolites({
    nad_c: -1.0,
    nadph_c: -1.0,
    nadh_c: 1.0,
    nadp_c: 1.0
})

NADTRHD.gene_reaction_rule = '(Avin_01840 and Avin_01850 and Avin_01860)'

NAD_H2.annotation.update({'ec-code': ['1.6.1.1', '1.6.1.2'],
                        'bigg.reaction': 'NADTRHD',
                        'kegg.reaction': 'R00112',
                        'metanetx.reaction': 'MNXR101898',
                        'seed.reaction': 'rxn00083'                        
                      })


model.add_reactions([NADTRHD])

## Set Glutamate Dehydrogenase to zero as is only used under high NH<sub>4</sub> concentration 
Kleiner and Kleinschmidt 1976  Journal of bacterialogy and Kleiner 1975 Arch microbiology


In [60]:
model.reactions.get_by_id("GLUDxi").upper_bound = 1
model.reactions.get_by_id("GLUDxi").lower_bound = 0


model.reactions.get_by_id("GLUDy").upper_bound = 1
model.reactions.get_by_id("GLUDy").lower_bound = 0

#Also pick one Glutamate synthetase (NADPH) for simplicity

model.reactions.get_by_id("GLUSx").upper_bound = 0
model.reactions.get_by_id("GLUSx").lower_bound = 0

#Malic enzyme to NADH for simpicty 
model.reactions.get_by_id("ME2").upper_bound = 0
model.reactions.get_by_id("ME2").lower_bound = 0

#Malic enzyme to NADH for simpicty 
model.reactions.get_by_id("SSCOARy").upper_bound = 0
model.reactions.get_by_id("SSCOARy").lower_bound = 0

## Quninone is the only inter membrane aromatic electron carrier in A.v. 

There is no biochemical evidence of menaquinone in the membrane only Q8. 

See:
Wong and Maier 1984 (https://jb.asm.org/content/159/1/348)
Jurtshuk et al 1969 (J Bac 98:3 1120-1127. L-malate oxidation)
Jones et al. 1966 Electron trasnport in Azotobacter vinelandii (BBA 113:3 467-481)



In [61]:
model.reactions.get_by_id("HYD2pp").upper_bound = 0
model.reactions.get_by_id("HYD2pp").lower_bound = 0

model.remove_reactions
model.reactions.get_by_id("HYD3pp").upper_bound = 0
model.reactions.get_by_id("HYD3pp").lower_bound = 0

model.reactions.get_by_id("NADH10").upper_bound = 0
model.reactions.get_by_id("NADH10").lower_bound = 0

model.reactions.get_by_id("NADH9").upper_bound = 0
model.reactions.get_by_id("NADH9").lower_bound = 0

model.reactions.get_by_id("NADPHQR3").upper_bound = 0
model.reactions.get_by_id("NADPHQR3").lower_bound = 0

model.reactions.get_by_id("NADPHQR4").upper_bound = 0
model.reactions.get_by_id("NADPHQR4").lower_bound = 0

model.reactions.get_by_id("FRD2").upper_bound = 0
model.reactions.get_by_id("FRD2").lower_bound = 0

model.reactions.get_by_id("MDH3").upper_bound = 0
model.reactions.get_by_id("MDH3").lower_bound = 0

model.reactions.get_by_id("DHORD5").upper_bound = 0
model.reactions.get_by_id("DHORD5").lower_bound = 0

model.reactions.get_by_id("CYTBD2pp").upper_bound = 0
model.reactions.get_by_id("CYTBD2pp").lower_bound = 0


menaquinone_rxns =["HYD2pp", "HYD3pp", "NADH10", "NADH9", "NADPHQR3","NADPHQR4","FRD2", "MDH3", "DHORD5", "CYTBD2pp"]

In [62]:
menaquinone_rxns =["HYD2pp", "HYD3pp", "NADH10", "NADH9", "NADPHQR3","NADPHQR4","FRD2", "MDH3", "DHORD5", "CYTBD2pp"]
model.remove_reactions(reactions = menaquinone_rxns)

In [63]:
#model.reactions.get_by_id("CYTBD2pp")

## Set Succinate dehydrogenase to quinone not FADH

In [64]:
model.reactions.get_by_id("SUCD1").upper_bound = 0
model.reactions.get_by_id("SUCD1").lower_bound = 0


model.reactions.get_by_id("FRD").upper_bound = 0
model.reactions.get_by_id("FRD").lower_bound = 0


model.reactions.get_by_id("FRD3").upper_bound = 0
model.reactions.get_by_id("FRD3").lower_bound = 0

# Set NAD:FAD oxidoreduxtase to zero as its a bad annotation for the Fix reaction

In [65]:
model.reactions.get_by_id("NADFADOR").upper_bound = 0
model.reactions.get_by_id("NADFADOR").lower_bound = 0

model.reactions.get_by_id("NADHNQR").upper_bound = 0
model.reactions.get_by_id("NADHNQR").lower_bound = 0

## Remove hydrogen producing 3-hydroxyadiapyl-CoA dehydrogenase (keeping NAD dependant)

In [66]:
model.reactions.get_by_id("HADPCOADH").upper_bound = 0
model.reactions.get_by_id("HADPCOADH").lower_bound = 0

## Removing quinone loops

In [67]:

model.reactions.get_by_id("GLYCTO2").upper_bound = 0
model.reactions.get_by_id("GLYCTO2").lower_bound = 0


model.reactions.get_by_id("G3PD5").upper_bound = 0
model.reactions.get_by_id("G3PD5").lower_bound = 0



model.reactions.get_by_id("NADPHQR2").upper_bound = 0
model.reactions.get_by_id("NADPHQR2").lower_bound = 0


model.reactions.get_by_id("PROD3").upper_bound = 0
model.reactions.get_by_id("PROD3").lower_bound = 0

## Reversible transport ends in futile cycle so make irreversible 

In [68]:

# Acetate
model.reactions.get_by_id("ACt2rpp").upper_bound = 1000
model.reactions.get_by_id("ACt2rpp").lower_bound = 0


#Acetoacetate
model.reactions.get_by_id("ACACt2pp").upper_bound = 1000
model.reactions.get_by_id("ACACt2pp").lower_bound = 0

#pyruvate

model.reactions.get_by_id("PYRt2rpp").upper_bound = 1000
model.reactions.get_by_id("PYRt2rpp").lower_bound = 0

#Glyoxylate
model.reactions.get_by_id("GLXtpp").upper_bound = 1000
model.reactions.get_by_id("GLXtpp").lower_bound = 0

#malonate 

model.reactions.get_by_id("MALONpp").upper_bound = 1000
model.reactions.get_by_id("MALONpp").lower_bound = 0

#alphaketogluterate

model.reactions.get_by_id("AKGt2rpp").upper_bound = 1000
model.reactions.get_by_id("AKGt2rpp").lower_bound = 0

#fructose FUCtpp
model.reactions.get_by_id("FUCtpp").upper_bound = 1000
model.reactions.get_by_id("FUCtpp").lower_bound = 0

#butyrate
model.reactions.get_by_id("BUTt2rpp").upper_bound = 1000
model.reactions.get_by_id("BUTt2rpp").lower_bound = 0

#proline 
model.reactions.get_by_id("PROt2rpp").upper_bound = 1000
model.reactions.get_by_id("PROt2rpp").lower_bound = 0

#Glutamate 
model.reactions.get_by_id("GLUt2rpp").upper_bound = 1000
model.reactions.get_by_id("GLUt2rpp").lower_bound = 0

#glycolate 
model.reactions.get_by_id("GLYCLTt2rpp").upper_bound = 1000
model.reactions.get_by_id("GLYCLTt2rpp").lower_bound = 0

#lactate 
model.reactions.get_by_id("D_LACt2pp").upper_bound = 1000
model.reactions.get_by_id("D_LACt2pp").lower_bound = 0

#fumurate

model.reactions.get_by_id("FUMt1pp").upper_bound = 1000
model.reactions.get_by_id("FUMt1pp").lower_bound = 0

## Fructose uptake is annotated twice
Genes Avin_12210 and Avin_12190 which are FruA and FruB respectivly are in FRUptspp and FRUpts2pp which make Fructose-1-phophate and fructose-6-phosphate. Since there is no evidence of 6-phosphate transport mechanisms and ecoli FruAB a Fructose1-phosphate producing we will not use FRUpts2pp

In [69]:
model.reactions.get_by_id("FRUpts2pp").upper_bound = 0
model.reactions.get_by_id("FRUpts2pp").lower_bound = 0

### PFK gene reaction rule includes FruK 

In [70]:
#PFK2 - Avin_23410 FRUK - Avin_12200
model.reactions.get_by_id("PFK").gene_reaction_rule = '(Avin_23410)'

## Remove Fructose 6 phosphate aldoase to direct flux to PFK
left over from  ecoli model not enough evidencince to suppport use in Av

In [71]:
 
model.reactions.get_by_id("F6PA").upper_bound = 0
model.reactions.get_by_id("F6PA").lower_bound = 0

## Pentose phosphate modification

In [72]:
 
model.reactions.get_by_id("FBA3").upper_bound = 0
model.reactions.get_by_id("FBA3").lower_bound = 0

model.reactions.get_by_id("PFK_3").upper_bound = 0
model.reactions.get_by_id("PFK_3").lower_bound = 0

In [73]:
model.reactions.PFK

Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x01e2a26e1208
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate --> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,(Avin_23410)
Lower bound,0.0
Upper bound,999999.0


## remove ATP syntahsae that connects cytsol to extracellular

In [74]:

model.reactions.get_by_id("ATPS4r").upper_bound = 0
model.reactions.get_by_id("ATPS4r").lower_bound = 0


## Glucose uptake 

In [75]:
##Glucose uptake was shown to mostly be through gluP (Avin_04150, GLCt2pp) by Quiroz-Rocha et al 
#https://doi.org/10.1038/s41598-017-00980-5

#set pyrophosphate dependant trasnport (GLCptspp) to zero

model.reactions.get_by_id("GLCptspp").upper_bound = 0
model.reactions.get_by_id("GLCptspp").lower_bound = 0

## Determine new amonut of reactions and genes

In [76]:
print('Genes:', len(model.genes))
print('Reations:', len(model.reactions))
print('Metabolites:', len(model.metabolites))

Genes: 1300
Reations: 2466
Metabolites: 2003


## Make sure all reaction bounds are between -1000 and 1000

In [77]:
#limit reaction bounds to -1000 and 1000
for reaction in model.reactions:
    if reaction.lower_bound < -1000:
        reaction.lower_bound = -1000
    if reaction.upper_bound > 1000:
        reaction.upper_bound = 1000
        
        
#then check 
for reaction in model.reactions:
    if (reaction.lower_bound < -1000
        or reaction.upper_bound > 1000):
        print(reaction.name)
        print(reaction.lower_bound)
        print(reaction.upper_bound)

## Set Nitrogen (N2) lower bounds to -1000

In [78]:
model.reactions.get_by_id("EX_n2_e").upper_bound = 1000
model.reactions.get_by_id("EX_n2_e").lower_bound = -1000

In [79]:
model.id = "iAA1300"
model.id

'iAA1300'

## Save model with name and number of genes

In [80]:
save_json_model(model, "../Data/Models/iAA1300.json")
write_sbml_model(model, "../Data/Models/iAA1300.xml")